## 简答题
1. 如果训练集有100万个实例，训练决策树（无约束）大致的深度是多少？
2. 通常来说，子节点的基尼杂质是高于还是低于其父节点？是通常更高/更低？还是永远更高/更低？
3. 如果决策树过拟合训练集，减少max_depth是否为一个好主意？
4. 如果决策树对训练集欠拟合，尝试缩放输入特征是否为一个好主意？
5. 如果在给定的训练集上训练决策树需要一个小时，那么如果将特征数量变为两倍，训练大约需要多少时间？
6. 如果在包含100万个实例的训练集上训练决策树需要一个小时，那么在包含1000万个实例的训练集上训练决策树，大概需要多长时间？提示：考虑CART算法的计算复杂度。

In [ ]:
# 1：无约束的决策树会持续分裂，直到所有叶节点的实例 “纯”（即每个叶节点仅含一个实例，或所有实例属于同一类）
# 2：通常更低，且永远更低
# 3：过拟合的决策树因过度复杂（如深度过大、叶节点过多）而 “记住” 训练集噪声，泛化能力差。减少max_depth会限制树的复杂度，防止过度分裂，属于有效的正则化手段，能显著降低过拟合风险。
# 4：决策树的分裂基于特征的 “阈值判断”（如 “x>5”），与特征的尺度无关（例如 “x=5 米” 和 “x=500 厘米” 对分裂无影响）。缩放特征（如标准化、归一化）对决策树的结构和性能无任何影响
# 5：若特征数量从 m 变为 2m，其他条件不变，训练时间与 m 成正比，因此时间变为原来的 2 倍
# 6：CART 算法复杂度为 O (n・m・log (n))，假设特征数 m 不变，时间与 n・log (n) 成正比。
# 100 万实例（n₁=10⁶）：log₂(10⁶)≈20，因此 n₁・log (n₁)≈10⁶×20
# 1000 万实例（n₂=10⁷）：log₂(10⁷)≈23.25，因此 n₂・log (n₂)≈10⁷×23.25
# 时间比例为（10⁷×23.25）/（10⁶×20）≈10×1.16≈11.6，因此训练时间约为11.6 小时（大致 10-12 小时）。

## 编程题

1. 为 新月形 数据集训练并微调一棵决策树。

a. 使用make_moons(n_samples=10000，noise=0.4)生成一个 新月形 数据集。

b. 使用train_test_split()拆分训练集和测试集。

c. 使用交叉验证的网格搜索（在GridSearchCV类的帮助下）为Decision-TreeClassifier找到适合的超参数值。提示：尝试max_leaf_nodes的多种值。

d. 使用超参数对整个训练集进行训练，并测量模型在测试集上的性能。你应该得到约85%～87%的准确率。

2. 按照以下步骤种植森林。

a.继续之前的练习，生产1000个训练集子集，每个子集包含随机挑选的100个实例。提示：使用Scikit-Learn的ShuffleSplit类来实现。

b.使用前面得到的最佳超参数值，在每个子集上训练一棵决策树。在测试集上评估这1000棵决策树。因为训练集更小，所以这些决策树的表现可能比第一棵决策树要差一些，只能达到约80%的精度。

c.见证奇迹的时刻到了。用每个测试集实例，生成1000棵决策树的预测，然后仅保留次数最频繁的预测［可以使用SciPy的mode()函数］。这样你在测试集上可获得大多数投票的预测结果。

d.评估测试集上的这些预测，你得到的准确率应该比第一个模型更高（高出0.5%～1.5%）。恭喜，你已经训练出了一个随机森林分类器！

In [7]:
import numpy as np
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split, ShuffleSplit
from sklearn.metrics import accuracy_score
# a. 生成新月形数据集（10000个样本，噪声0.4）
X, y = make_moons(n_samples=10000, noise=0.4, random_state=42)

# b. 拆分训练集（80%）和测试集（20%）
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [5]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV

# 定义决策树模型
dtc = DecisionTreeClassifier(random_state=42)

# 定义超参数候选值（重点测试max_leaf_nodes）
param_grid = {
    'max_leaf_nodes': [None, 2, 5, 10, 20, 30, 50, 100, 200],  # 叶节点数量
    'min_samples_split': [2, 5, 10],  # 拆分节点所需最小样本数（辅助调优）
    'max_depth': [None, 5, 10, 20]  # 树的最大深度（辅助调优）
}

# 网格搜索（5折交叉验证）
grid_search = GridSearchCV(
    estimator=dtc,
    param_grid=param_grid,
    cv=5,
    scoring='accuracy',
    n_jobs=-1  # 利用所有CPU核心加速
)

# 在训练集上拟合
grid_search.fit(X_train, y_train)

# 输出最优超参数
print( grid_search.best_params_)
print( grid_search.best_score_)

{'max_depth': None, 'max_leaf_nodes': 20, 'min_samples_split': 2}
0.858625


In [6]:
# c. 用最优超参数训练模型
best_dtc = grid_search.best_estimator_

# d. 在测试集上评估
test_accuracy = best_dtc.score(X_test, y_test)
test_accuracy

0.87

In [9]:
# a. 生成1000个训练集子集，每个包含100个随机挑选的实例
from scipy.stats import mode
n_trees = 1000
n_instances = 100

# 使用ShuffleSplit生成1000个随机子集，每个子集大小为100
rs = ShuffleSplit(n_splits=n_trees, train_size=n_instances, random_state=42)

# b. 在每个子集上训练决策树，并评估性能
trees = []
scores = []

# 使用之前找到的最佳超参数
best_params = {'max_leaf_nodes': 20, 'random_state': 42}

for train_index, _ in rs.split(X_train):
    X_subset = X_train[train_index]
    y_subset = y_train[train_index]
    tree = DecisionTreeClassifier(**best_params)
    tree.fit(X_subset, y_subset)
    trees.append(tree)

    # 在测试集上评估
    y_pred = tree.predict(X_test)
    scores.append(accuracy_score(y_test, y_pred))

# 打印单个决策树的平均准确率
print(f"单个决策树的平均准确率: {np.mean(scores):.4f}")
print(f"单个决策树的准确率标准差: {np.std(scores):.4f}")

# c. 生成多数投票预测结果
# 初始化预测结果数组，形状为(测试集大小, 决策树数量)
all_predictions = np.empty((len(X_test), n_trees))

# 收集所有决策树的预测结果
for i, tree in enumerate(trees):
    all_predictions[:, i] = tree.predict(X_test)

# 对每个测试实例，取多数投票结果
# 使用mode函数获取最频繁的预测值
y_pred_majority_votes, _ = mode(all_predictions, axis=1)

# d. 评估多数投票的准确率
# 将结果转换为一维数组
y_pred_majority_votes = y_pred_majority_votes.reshape(-1)

# 计算准确率
ensemble_accuracy = accuracy_score(y_test, y_pred_majority_votes)
print(f"随机森林（多数投票）的准确率: {ensemble_accuracy:.4f}")
print(f"提升幅度: {(ensemble_accuracy - np.mean(scores)):.4f}")

单个决策树的平均准确率: 0.8012
单个决策树的准确率标准差: 0.0250
随机森林（多数投票）的准确率: 0.8720
提升幅度: 0.0708
